In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the dataset from JSON file
file_path = '/Users/aqib/Desktop/YEAR3/project/2500.json'
with open(file_path, 'r') as f:
    lines = f.readlines()

# Each line in the file is a separate JSON object, handle potential empty lines
data = []
for line in lines:
    line = line.strip()
    if line:
        data.append(json.loads(line))

# Convert to DataFrame
df = pd.DataFrame(data)

# Extract review texts and labels
texts = df['reviewText'].values
labels = df['overall'].values

# Preprocess labels
labels = labels - 1  # Adjust labels to be 0-4 for a 5-class classification

# Tokenize and pad sequences
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
maxlen = 100
data = pad_sequences(sequences, maxlen=maxlen)

# Build the model
model = Sequential()
model.add(Embedding(10000, 128, input_length=maxlen))
model.add(SpatialDropout1D(0.3))  # Increased dropout
model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model.add(LSTM(64, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Callbacks for early stopping and saving the best model
callbacks = [
    EarlyStopping(monitor='val_loss', patience=2, min_delta=0.0001),
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')
]

# Train the model
history = model.fit(data, labels, epochs=10, batch_size=32, validation_split=0.2, callbacks=callbacks)

# Evaluate the model
accr = model.evaluate(data, labels)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0], accr[1]))

2024-07-11 13:21:38.991015: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


37/37 ━━━━━━━━━━━━━━━━━━━━ 12s 198ms/step - accuracy: 0.5530 - loss: 1.3632 - val_accuracy: 0.6575 - val_loss: 1.0094
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 7s 192ms/step - accuracy: 0.6511 - loss: 1.0482 - val_accuracy: 0.6575 - val_loss: 1.0088
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 189ms/step - accuracy: 0.6340 - loss: 1.0939 - val_accuracy: 0.6575 - val_loss: 0.9952
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 7s 184ms/step - accuracy: 0.6305 - loss: 1.0886 - val_accuracy: 0.6644 - val_loss: 0.9555
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 7s 187ms/step - accuracy: 0.6492 - loss: 0.9075 - val_accuracy: 0.6712 - val_loss: 0.9541
Epoch 6/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 7s 188ms/step - accuracy: 0.7037 - loss: 0.7526 - val_accuracy: 0.6267 - val_loss: 0.9991
Epoch 7/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 8s 203ms/step - accuracy: 0.7948 - loss: 0.5786 - val_accuracy: 0.6438 - val_loss: 1.0595
46/46 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.8527 - loss: 0.3960
Test set
  Loss: 0.550
  Accuracy: 0.80